In [ ]:
import os
import shutil
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

## Limpar Pastas que não serão usadas mais

In [ ]:
conjuntos = ['treinamento', 'teste', 'validacao']
base_path = '/home/bernardo/TP2-ICV/dados/processed'
pastas_excluir = ['cleanImage', '?']

for conjunto in conjuntos:
    conjunto_path = os.path.join(base_path, conjunto)
    for nome in os.listdir(conjunto_path):
        full_path = os.path.join(conjunto_path, nome)
        if nome in pastas_excluir and os.path.isdir(full_path):
            print(f"Removendo pasta: {full_path}")
            shutil.rmtree(full_path)

## Carregar os datasets a partir das pastas

In [2]:
train_dir = 'dados/processed/treinamento'
val_dir   = 'dados/processed/validacao'
test_dir  = 'dados/processed/teste'

batch_size = 32
img_size = (36, 50)

train_ds = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    color_mode='grayscale',
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    val_dir,
    label_mode='categorical',
    color_mode='grayscale',
    image_size=img_size,
    batch_size=batch_size
)

test_ds = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    color_mode='grayscale',
    image_size=img_size,
    batch_size=batch_size
)

Found 47868 files belonging to 36 classes.


2025-06-22 09:58:02.915700: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Found 5994 files belonging to 36 classes.
Found 5994 files belonging to 36 classes.


In [ ]:
model = models.Sequential([
    layers.Input(shape=img_size + (1,)),
    layers.Rescaling(1./255), 

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_ds.element_spec[1].shape[1], activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinamento
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)

Epoch 1/15
 158/1496 ━━━━━━━━━━━━━━━━━━━━ 4:31 203ms/step - accuracy: 0.0702 - loss: 3.4319

In [ ]:
# Avaliação
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest accuracy: {test_acc:.4f}")

# Gráficos
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Treinamento')
plt.plot(epochs_range, val_acc, label='Validação')
plt.title('Acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Treinamento')
plt.plot(epochs_range, val_loss, label='Validação')
plt.title('Erro (Loss)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()